In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'Material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Package Quantity"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/Delete_Motion_Max_Min_temp_date.sql', params=params)
# df=df[df['value']=='n/a']
print(len(df))
df.head(1)

497187


attributes snake_case_name                                             name  \
0   Material        material  1-Light Faux Wood Layered Table Lamp in Natural   

  external_id  \
0    42588269   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
0  You will bring a functional and natural-looking touch to your home décor with the Faux Wood Layered Table Lamp. The lamp's lengthy cord offers easy placement, and its rectangular shade adds character to any room. The Faux Wood Layered Table Lamp with Shade will add a functional and natural-looking touch to your home décor Features a unique faux wood style Includes a rectangle hardback linen shade Adds a distinctive and rustic look to any room Long cord length allows easy placement 3-way switch Rectangular shade brings a distinctive look to any room Natural, resin finish provides a bucolic look Lamp is made of 70% polyresin, 20% fabric, and 10% metal Hardback linen shade Uses one 100-watt bulb (not included) Plugs into any standard AC outlet Measures 23.25\" H Rectangular shade measures 8\" L x 8\" W x 10.5\" H and 17\" L x 17\" W x 10.5\" H Cord measures 72\" L Weighs 9.7 lb. CUL listed 1-year manufacturer's warranty Imported Model number L31878BBS   

  curated_date  customers_id buckets                  value   family_id  \
0   2021-08-16             5   Lamps  ["Linen","Polyresin"]  1042588269   

  resolution  
0   standard

In [12]:
regex_pattern_final_regular = r"(?i)(sea.?foam)|()"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x)))
regex_pattern_final_regular = r"(?i)(foam)|()"
df['value_matches'] = df['value'].apply(lambda x: re_extract(regex_pattern_final_regular, str(x)))

In [6]:
df[df['external_id'].astype(str)=='40507439']

attributes snake_case_name  \
352088   Material        material   

                                                        name external_id  \
352088  HiEnd Accents Arlington 84-Inch Window Curtain Panel    40507439   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     long_desc  \
352088  Create a soothing ambiance in your windows with the HiEnd Accents Arlington Window Curtain Panel. The paisley printed panel dresses your window in a luxurious rustic style for a sophisticated look in your living space. Panel features a paisley print in hues of beige and sea foam blue 100% chenille Dry clean only Panel hangs on a rod pocket Panels are sold individually and each measures 48\" W Rod pocket measures 1.5\" diameter Imported Coordinate with HiEnd Accents Arlington accessories for a complete look (each sold separately)   

       curated_date  customers_id   buckets                value   family_id  \
352088   2021-01-28             5  Curtains  ["Chenille","Foam"]  1040507439   

       resolution     matches value_matches  
352088      rules  [sea foam]        [Foam]

In [15]:
foam=df[(df['value_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
foam[curation_col]=''
foam_new=foam[['external_id',curation_col]]
foam

attributes snake_case_name  \
326793   Material        material   
336833   Material        material   
343109   Material        material   
346930   Material        material   
352088   Material        material   
353189   Material        material   
353190   Material        material   
371794   Material        material   
383039   Material        material   
383040   Material        material   
393631   Material        material   
393632   Material        material   
393634   Material        material   
420299   Material        material   
420300   Material        material   
420316   Material        material   
420317   Material        material   
420318   Material        material   

                                                                         name  \
326793                       Nanshing Roxanne King Comforter Set in Grey/Blue   
336833                      Nanshing Roxanne Queen Comforter Set in Grey/Blue   
343109                   Donna Sharp Summer Surf Standard Pillow Sham in Blue   
346930            Nanshing Roxanne California King Comforter Set in Grey/Blue   
352088                   HiEnd Accents Arlington 84-Inch Window Curtain Panel   
353189                         Madison Park Baxter 7-Piece King Comforter Set   
353190                        Madison Park Baxter 7-Piece Queen Comforter Set   
371794                   Coastal Life Nantucket European Pillow Sham in Taupe   
383039               HiEnd Accents Arlington Velvet King Duvet Cover in Green   
383040              HiEnd Accents Arlington Velvet Queen Duvet Cover in Green   
393631             Lush Décor Whale 4-Piece Reversible Twin Quilt Set in Navy   
393632       Lush Décor Whale 5-Piece Reversible Full/Queen Quilt Set in Navy   
393634                          Lush Décor Whale Sherpa Throw Blanket in Navy   
420299                  Seaside Resort Flamingo Print Queen Sheet Set in Pink   
420300  Seaside Resort Flamingo Print Standard Pillowcases in Pink (Set of 2)   
420316                 Seawaves Reversible Full/Queen Quilt Set in Blue/White   
420317                       Seawaves Reversible King Quilt Set in Blue/White   
420318                       Seawaves Reversible Twin Quilt Set in Blue/White   

       external_id  \
326793    67645411   
336833    67645008   
343109    69582078   
346930    67645923   
352088    40507439   
353189    42416098   
353190    42416081   
371794    60736000   
383039    40508481   
383040    40508511   
393631    62646963   
393632    62646970   
393634    62647106   
420299    67169498   
420300    67169511   
420316    66292203   
420317    66292210   
420318    66292197   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [8]:
# df_new=df[df['resolution'].astype(str)=='rules']
# df_new[(df_new['curated_date'].astype(str)<'2021-01-01')] 

In [46]:
utensil=df_new[df_new['buckets'].astype(str)=='Utensil Holders/Crocks']
utensil[curation_col]=''
utensil=utensil[['external_id',curation_col]]
# utensil

In [45]:
frying_pan_wipe=df_new[df_new['buckets'].astype(str)=='Frying Pans & Skillets']
frying_pan_wipe[curation_col]=''
frying_pan_wipe=frying_pan_wipe[['external_id',curation_col]]
# frying_pan_wipe

In [ ]:
error----Switch Max and Min when doing length and width

In [35]:
# gc = gcloud.Gsheets()

In [36]:
# gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [16]:
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-foam - Wipe.csv',index=False) 

In [17]:
looks_good('Bed Bath & Beyond', attribute, df, foam_new)